<a href="https://colab.research.google.com/github/SardarAmmer/Report-Generation/blob/main/Projectwise_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>